# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [ ]:
# Imports here
# ChatGPT was used at certain points to help with code troubleshooting
# Some GitHub repositories were consulted when presented with issues such as nouran551 and CaterinaBi
# Individual code is my own

import torch
import numpy as np
from torch import nn
from torch import optim
import torch.nn.functional as F

from torchvision import datasets, transforms, models

import json

from collections import OrderedDict

import time

import numpy as np

from PIL import Image

import matplotlib.pyplot as plt

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [ ]:
data_dir = 'flowers'
print(data_dir)
training_dir = data_dir + '/training'
print(train_dir)
validation_dir = data_dir + '/validation'
print(valid_dir)
testing_dir = data_dir + '/testing'
print(test_dir)

In [ ]:
from torchvision import datasets, transforms

# Transforms
common_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_transforms = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    common_transforms
])

valid_transforms = common_transforms
test_transforms = common_transforms

# Datasets
data_dir = '/workspace/home/aipnd-project/flowers'  # replace with your actual dataset directory

train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)
valid_data = datasets.ImageFolder(data_dir + '/valid', transform=valid_transforms)

# Dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=32)

# Image datasets and dataloaders
image_datasets = [train_data, valid_data, test_data]
dataloaders = [trainloader, validloader, testloader]

# Print dataset information
for dataset, loader in zip(image_datasets, dataloaders):
    print(dataset)
    print(loader)


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
# loads in mapping from category label to category name
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

    # prints json file
    print(cat_to_name)

no_output_categories = len(cat_to_name)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [ ]:
# TODO: Build and train your network
# The link for the pre-trained network is dead. Googling PyTorch and VGG though brings up VGG16 
# pre-trained network VGG16 will be used

# Having some issues with models being recognized as torchvision.models so importing torchvision.model as models
import torchvision.models as models

# VGG16 will use 4096 hidden units for a fully connected first layer
hidden_units = 4096

# As indicated above VGG16 will be used
# UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
model = models.vgg16_bn(weights=True)

# The weights of the pretrained model will be frozen to limit data usage on the GPU network
for param in model.parameters():
    param.requires_grad = False

model.classifier

In [ ]:
# Creating a new classifier for VGG16
# Replacing original classifier for requirements of this program
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(25088, hidden_units)),
                          ('relU', nn.ReLU()),
                          ('dropout01', nn.Dropout(0.05)),
                          ('fch2', nn.Linear(hidden_units, no_output_categories)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))

model.classifier = classifier

model.classifier

In [ ]:
# This will set the training parameters.
import torch
import torch.optim as optim
import torch.nn as nn
import time

# Set the training parameters
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f'The device in use is {device}.\n')

# Training parameters
epochs = 5
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
criterion = nn.NLLLoss()
print_every = 20

running_loss = running_accuracy = 0
validation_losses, training_losses = [], []

# Training process
for e in range(epochs):
    batches = 0
    model.train()
    
    for images, labels in train_loader:
        start = time.time()
        batches += 1

        # Move images to GPU
        images, labels = images.to(device), labels.to(device)

        # Forward pass and calculate loss
        log_ps = model.forward(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()

        # Metric calculation
        ps = torch.exp(log_ps)
        top_ps, top_class = ps.topk(1, dim=1)
        matches = (top_class == labels.view(*top_class.shape)).type(torch.FloatTensor)
        accuracy = matches.mean()

        # Metric tracker
        optimizer.zero_grad()
        running_loss += loss.item()
        running_accuracy += accuracy.item()

        # Model validation and printing metrics
        if batches % print_every == 0:
            end = time.time()
            training_time = end - start
            start = time.time()

            # Validation metrics
            validation_loss = 0
            validation_accuracy = 0

            model.eval()
            with torch.no_grad():
                for images, labels in valid_loader:
                    images, labels = images.to(device), labels.to(device)
                    log_ps = model.forward(images)
                    loss = criterion(log_ps, labels)
                    ps = torch.exp(log_ps)
                    top_ps, top_class = ps.topk(1, dim=1)
                    matches = (top_class == labels.view(*top_class.shape)).type(torch.FloatTensor)
                    accuracy = matches.mean()

                    # Update validation metrics
                    validation_loss += loss.item()
                    validation_accuracy += accuracy.item()

            # Track training progress
            end = time.time()
            validation_time = end - start
            validation_losses.append(running_loss / print_every)
            training_losses.append(validation_loss / len(valid_loader))

            # Print metrics
            print(f'Epoch {e+1}/{epochs} | Batch {batches}')
            print(f'Running Training Loss: {running_loss / print_every:.3f}')
            print(f'Running Training Accuracy: {running_accuracy / print_every * 100:.2f}%')
            print(f'Validation Loss: {validation_loss / len(valid_loader):.3f}')
            print(f'Validation Accuracy: {validation_accuracy / len(valid_loader) * 100:.2f}%')

            # Reset metrics and turn on training mode
            running_loss = running_accuracy = 0
            model.train()


## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Do validation on the test set
# TODO: Do validation on the test set
test_accuracy = 0

# Pring the time it will take 
start_time = time.time()
print('VGG16 Initialized.')

# Defines the test conditions
for images,labels in test_loader:
    model.eval()
    images,labels = images.to(device),labels.to(device)
    log_ps = model.forward(images)
    ps = torch.exp(log_ps)
    top_ps,top_class = ps.topk(1,dim=1)
    matches = (top_class == labels.view(*top_class.shape)).type(torch.FloatTensor)
    accuracy = matches.mean()
    test_accuracy += accuracy

# Determine total time taken to perform
end_time = time.time()
print('Validation ended.')
validation_time = end_time - start_time
print('Validation time: {:.0f}m {:.0f}s'.format(validation_time / 60, validation_time % 60))
    
# Print the test accuracy. Target is ~70%
print(f'Test Accuracy: {test_accuracy/len(test_loader)*100:.2f}%')

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 
import os

def save_model(trained_model, hidden_units, output_units, destination_directory, model_arch, class_to_idx):
    # Define model's checkpoint
    model_checkpoint = {
        'model_arch': model_arch,
        'clf_input': 25088,
        'clf_output': output_units,
        'clf_hidden': hidden_units,
        'state_dict': trained_model.state_dict(),
        'model_class_to_index': class_to_idx,
    }
    
    # Save model in the specified directory or current directory
    save_path = destination_directory or os.getcwd()
    save_path = os.path.join(save_path, f"{model_arch}_checkpoint.pth")
    
    torch.save(model_checkpoint, save_path)
    print(f"{model_arch} successfully saved to {save_path}")

# Example usage
save_model(model, hidden_units, no_output_categories, destination_directory, 'vgg16_bn', class_to_idx)


## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
# Filepath to the checkpoint
checkpoint_filepath = 'vgg16_bn_checkpoint.pth'

def load_checkpoint(filepath, device):
    # Map location to GPU if available, else use CPU
    map_location = 'cuda' if device == 'gpu' and torch.cuda.is_available() else 'cpu'

    # Load the checkpoint
    checkpoint = torch.load(filepath, map_location=map_location)

    # Return relevant information
    return (
        checkpoint['model_arch'],
        checkpoint['clf_input'],
        checkpoint['clf_output'],
        checkpoint['clf_hidden'],
        checkpoint['state_dict'],
        checkpoint['model_class_to_index']
    )

# Load the checkpoint
model_arch, input_units, output_units, hidden_units, state_dict, class_to_idx = load_checkpoint(checkpoint_filepath, device)
model.load_state_dict(state_dict)


# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
# TODO: Process a PIL image for use in a PyTorch model
# Calling a sample image from the test assets
practice_flower = './flowers/test/100/image_07896.jpg'

def process_image(image):
    processed_image = Image.open(image).convert('RGB') 
    processed_image.thumbnail(size=(256,256))
    width, height = processed_image.size

    # Sets dimensions
    new_width,new_height = 224,224 
    left = (width - new_width)/2
    top = (height - new_height)/2
    right = (width + new_width)/2
    bottom = (height + new_height)/2
    processed_image = processed_image.crop((left, top, right, bottom))

    # Converts to tensor using the transforms from the code above.
    transf_tens = transforms.ToTensor()
    transf_norm = transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    tensor = transf_norm(transf_tens(processed_image))
    
    # Generate the numpy array
    np_processed_image = np.array(tensor)
    return np_processed_image
    

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    ax = ax or plt.subplots()[1]
    ax.tick_params(axis='both', which='both', bottom=False, top=False, left=False, labelbottom=False, labelleft=False)

    image = (image.transpose((1, 2, 0)) * np.array([0.229, 0.224, 0.225])) + np.array([0.485, 0.456, 0.406])
    image = np.clip(image, 0, 1)

    ax.imshow(image)
    return ax


# Displaying image

processed_image = Image.open(practice_flower)
processed_image

In [ ]:
# Displaying the preprocessed image
print(type(practice_flower))
processed_image = process_image(practice_flower)
print(type(processed_image))
imshow(processed_image)

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
import json

file = 'cat_to_name.json'

def class_to_label(file, classes):
    with open(file, 'r') as f:
        class_mapping = json.load(f)
        
    return [class_mapping[c] for c in classes]

idx_mapping = dict(map(reversed, class_to_idx.items()))

def predict(image_path, model, idx_mapping, topk, device):
    pre_processed_image = torch.from_numpy(process_image(image_path))
    pre_processed_image = torch.unsqueeze(pre_processed_image, 0).to(device).float()
    
    model.to(device).eval()
    
    log_ps = model.forward(pre_processed_image)
    ps, top_idx = torch.exp(log_ps).topk(topk, dim=1)
    
    classes = [idx_mapping[x] for x in top_idx.tolist()[0]]
    
    return ps.tolist()[0], classes

def print_predictions(probabilities, classes, image, category_names=None):
    print(image)
    
    labels = class_to_label(category_names, classes) if category_names else classes
    
    for i, (ps, ls, cs) in enumerate(zip(probabilities, labels, classes), 1):
        print(f'{i}) {ps * 100:.2f}% {ls.title()} | Class No. {cs}' if category_names else f'{i}) {ps * 100:.2f}% Class No. {cs}')
    
    print('') 

probabilities, classes = predict(practice_img, model, idx_mapping, 5, device)
print_predictions(probabilities, classes, practice_img.split('/')[-1], file)


## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes
# displays an image along with the top 5 classes
imshow(process_image(practice_img))

# uses matplotlib for the task
plt.figure() # image
# defines probability axis
plt.barh(class_to_label(file,classes),width=probabilities)
plt.title('Model Predictions') # defines title
plt.gca().invert_yaxis() # defines y axis
plt.show() # shows diagram